In [2]:


import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import make_scorer, f1_score

#from xgboost import XGBRegressor

In [3]:
mainDF = pd.read_csv(r'C:\Users\raadr\OneDrive\Desktop\AflAnalysis-\data\interim\proH&AData.csv')
mainDF.columns.tolist()
#drop null values
mainDF = mainDF.dropna()

FileNotFoundError: [Errno 2] No such file or directory: 'proH&AData.csv'

In [10]:
X = mainDF[['Year', 'Round', 'MaxTemp', 'MinTemp', 'Rainfall', 'Venue', 'HomeTeam', 'AwayTeam', 'Day', 'homePosition', 'homePoints', 'MinutesSinceMidnight','homePercentage', 'awayPosition']]
Y = mainDF['Attendance']

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)

In [12]:
#Normalise Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
model = GradientBoostingRegressor()
model.fit(X_train, y_train)

In [14]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results

In [ ]:
# Plot results
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Attendance')
plt.ylabel('Predicted Attendance')
plt.title('Actual Attendance vs Predicted Attendance')
plt.show()


In [ ]:
#plot feature importance
importance = model.feature_importances_
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

# Cat Boosting

In [19]:
from catboost import CatBoostRegressor

In [20]:
mainDF = pd.read_csv('proCatH&AData.csv')
mainDF.columns.tolist()
#drop null values
mainDF = mainDF.dropna()

In [21]:
categorical_features = ['Venue', 'HomeTeam', 'AwayTeam', 'Day', 'StartTime']
cbr = CatBoostRegressor(loss_function='RMSE', cat_features=categorical_features, random_state=42)

In [22]:
X = mainDF[['Year', 'Round', 'MaxTemp', 'MinTemp', 'Rainfall', 'Venue', 'HomeTeam', 'AwayTeam', 'Day', 'homePosition', 'homePoints', 'StartTime','homePercentage', 'awayPosition']]
Y = mainDF['Attendance']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)
#Normalise Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

# Define the scorer
scorer = make_scorer(mean_squared_error, greater_is_better=False)

# Define the parameter distributions for Random Search
param_dist = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1],
    'depth': [3, 6]
}

# Perform Randomized Search with cross-validation
random_search = RandomizedSearchCV(cbr, param_distributions=param_dist, n_iter=10, cv=5, scoring=scorer, n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)

# Print the best hyperparameters for Random Search
print("Random Search - Best Hyperparameters:", random_search.best_params_)
# Plot the results of the Randomized Search
results = pd.DataFrame(random_search.cv_results_)
plt.figure(figsize=(10, 6))
sns.boxplot(x='param_iterations', y='mean_test_score', data=results)
plt.xlabel('Iterations')
plt.ylabel('Mean Test Score (Negative MSE)')
plt.title('Randomized Search Results')
plt.show()

In [ ]:

# Specify the categorical features
categorical_features = ['Venue', 'HomeTeam', 'AwayTeam', 'Day', 'StartTime']

cbr.fit(X_train, y_train, cat_features=categorical_features)


In [26]:
y_pred = cbr.predict(X_test)

In [ ]:

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results

In [ ]:
# Plot results
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Attendance')
plt.ylabel('Predicted Attendance')
plt.title('Actual Attendance vs Predicted Attendance')
plt.show()


In [ ]:
%pip install lightgbm


In [31]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [33]:
X = mainDF[['Year', 'Round', 'MaxTemp', 'MinTemp', 'Rainfall', 'Venue', 'HomeTeam', 'AwayTeam', 'Day', 'homePosition', 'homePoints', 'StartTime','homePercentage', 'awayPosition']]
Y = mainDF['Attendance']

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.20, random_state=42)

In [ ]:
categorical_features = ['Venue', 'HomeTeam', 'AwayTeam', 'Day', 'StartTime']

for col in categorical_features:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')

train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features)
test_data = lgb.Dataset(X_test, label=y_test, reference = train_data)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 11,
    'learning_rate': 0.05,
}

booster = lgb.train(params, train_data, 100, valid_sets=[train_data, test_data])

y_pred = booster.predict(X_test)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Define the parameter distributions for Random Search
param_dist_lgb = {
    'num_leaves': [7, 15, 31],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 500],
    'max_depth': [-1, 3, 5, 7]
}

# Perform Randomized Search with cross-validation
random_search_lgb = RandomizedSearchCV(lgr, param_distributions=param_dist_lgb, n_iter=10, cv=5, scoring=scorer, n_jobs=-1, random_state=42)
random_search_lgb.fit(X_train, y_train)

# Print the best hyperparameters for Random Search
print("Random Search - Best Hyperparameters for LightGBM:", random_search_lgb.best_params_)

# Train the model with the best hyperparameters
best_lgr = random_search_lgb.best_estimator_
best_lgr.fit(X_train, y_train)

# Predict using the best model
y_pred_best = best_lgr.predict(X_test)

In [ ]:

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

# Calculate R-squared score
r2 = r2_score(y_test, y_pred)
print("R-squared:", r2)

In [ ]:
results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
results

In [ ]:
# Plot results
plt.scatter(y_test, y_pred)
plt.xlabel('Actual Attendance')
plt.ylabel('Predicted Attendance')
plt.title('Actual Attendance vs Predicted Attendance')
plt.show()
